## Introduction to boundary value problems

What is the shape of a hanging chain? We can make this question more precise. Suspend a
chain (or rope or cord) of length $L$ from its ends, which are fixed to the points $(x_a,y_a)$ and
$(x_b,y_b)$ in the $x$-$y$ plane. What is the curve $y(x)$ taken by the chain? To answer this question,
we first derive an ordinary differential equation satisfied by the function $y(x)$. The equation
can be solved numerically as a __boundary value problem__. With a boundary value problem, the
freely chosen data are specified at the boundaries (or endpoints) of the system. 
In contrast,
the freely chosen data for an _initial value problem_ consist of initial data—--typically the initial
positions and velocities. For the hanging chain, the boundary data (or boundary
conditions) consist of the relations $y_a = y(x_a)$ and $y_b = y(x_b)$.


From the slides, we find:
$$
        y_i = \frac{1}{2}(y_{i+1} + y_{i-1}) - \frac{k h^2 }{2}\sqrt{1 + (y_{i+1} - y_{i-1})^2/(2h)^2} \ .
$$

## Problem 1

Write a code that uses relaxation to solve the differential equation above. Use $k=5.0$ for the constant, 
and choose boundary conditions $y(0) = 0$ and $y(1) = 2$. For your trial solution, use the straight line $y_i = 2.0 \,x_i$. 
Experiment with different numbers of relaxation sweeps using a fixed number of nodes, say, $N=100$. 
Plot a graph of $y$ versus $x$ for various numbers of sweeps. About how many sweeps are required for curve to relax to the 
solution (that is -- when it stops changing)? 

**Array slicing:**
You may have written your code using a `for` loop, or similar control structure, to implement the relaxation sweeps. For 
example, you could write 

    for i in range(1,N):
         y[i] = 0.5*(y[i+1] + y[i-1]) + ...


With numpy's intrinsic indexing (also known as *slicing*), this loop can be replaced by a single statement
$$
y[1:-1] = 0.5*(y[:-2] + y[2:]) + ...
$$
Here, `y` is a numpy array with elements `y[0]`, `y[1]`,$\ldots$, `y[N]`. To understand what that means, consider these examples:

* The syntax `y[1:-1]` denotes the range of elements beginning with `y[1]`, and ending with `y[N-1]`. 
* The syntax `y[:-2]` is equivalent to `y[0:-2]`; it denotes the range of elements beginning with `y[0]` and ending with `y[N-2]`. 
* The syntax `y[2:]` is equivalent to `y[2:N+1]`; it denotes the range of elements beginning with `y[2]` and ending with `y[N]`. 

*Optional:* Your code will run _much_ faster if you use the single statement above, rather than the explicit loop. 

In [ ]:
####import statements go here

# define the number of steps/nodes, the array that will hold all the steps
N= ###FILL IN THE BLANK
y= ###FILL IN THE BLANK
y[0]= ###FILL IN THE BLANK, this is the initial condition on the left
y[-1]= ###FILL IN THE BLANK, this is the initial condition on the right

Compare the amount of time it takes to index with two different methods. Why does one take longer than the other?

In [ ]:
%%timeit
for i in range(N):
    y[1:-1]= 0.5 * (y[2:]+y[:-2])

In [ ]:
%%timeit
for j in range(N):
    for i in range(1, N-1):
        y[i]= (y[i+1]+y[i-1])/2

In [ ]:
#add import statements


# BVP for chain hanging. Equation is y''= k (1+y'^2)^0.5

#First: define the equation to solve. Note that y1 is i+1, y2 is i-1
def chain(y1, y2, h, k):
    yi = ####FILL IN THE BLANK    
    return yi

###########################
# Main code
###########################
nodeN= ####FILL IN THE BLANK # numbers of nodes
trialN= 300000+1 # How many trials
callPlot= [####FILL IN THE BLANK, trialN-1] # pick six other trial points, let the last be trialN-1

# Physics setup
x= np.linspace(0, 1, nodeN+1) # base of N, the number of steps 
h= x[1]-x[0] # increment to step through
k= ####FILL IN THE BLANK # spring const.
y= ####FILL IN THE BLANK # initial guess
py.plot(x, y, 'o-', label='starting condition') # plot the initial guess
 
for i in range(trialN):
    y[1:-1]= chain(y[2:], y[:-2], h, k)
    if i in callPlot:
        py.plot(x, y, 'o-', label= str(i)+'th step')

py.legend()
py.show()

## Controlling the error

How many relaxation sweeps are required before the solution stops changing significantly? How should you decide what constitutes 
a significant change? One way to quantify the change is to monitor the difference between successive "solutions". 
Let $y_i^{old}$ denote the solution obtained after $s-1$ sweeps, and $y_i$ denote the solution obtained after $s$ sweeps. 
The $L_1$ norm is the average of the absolute value of the difference:  
$$
        L_1 = \frac{1}{N} \sum_{i=0}^N |y_i - y_i^{old}| \ .
$$
This norm is sometimes called the taxicab, or Manhattan norm (can you figure out why?)

## Problem 2

Modify the code to monitor the $L_1$ norm of the difference between successive solutions. Have your code continue 
to carry out relaxation sweeps until the $L_1$ norm drops below some prescribed value $L_1^{max}$. You will need to experiment with 
your code to determine a reasonable value for $L_1^{max}$. Your goal is to have some assurance that at each node, your numerical solution is 
within, say, a fraction of a percent of the "ideal" $s\to\infty$ solution. 

In [ ]:
#import statements
import numpy as np
import pylab as py

# same user defined function from above, copy and paste your working code here. 
# you need to redefine it here to use it in this textbox
def chain(y1, y2, h, k):
    yi = ####FILL IN THE BLANK    
    return yi

nodeN= ####FILL IN THE BLANK IN H # numbers of nodes
trialN = ####FILL IN THE BLANK # max number of trials
L1 = np.zeros(trialN) # norm of errors
L1max = ####FILL IN THE BLANK, pick some sort of error

# Physics setup
x= np.linspace(0, 1, nodeN+1) # base of N, the number of steps 
h= x[1]-x[0] # increment to step through
k= ####FILL IN THE BLANK # spring const.
y= ####FILL IN THE BLANK # initial guess
 
 
for i in range(trialN):
    yprev = np.copy(y)
    y[1:-1]= chain(y[2:], y[:-2], h, k)
    L1[i] = np.mean(np.abs(yprev-y))
    if L1[i] <  L1max:
        Nmax = i
        print(str(Nmax) + ' steps needed to reach L1max < '+str(L1max))
        break

    
py.plot(np.arange(Nmax), L1[:Nmax], '.')
#py.yscale('log')
py.xlabel('relaxation step #')
py.ylabel('L1 change since previous step')
py.show()

Now, we can talk about investigating the length of the chain, aka coming at this problem from a different perspective when L isn't fixed and k is fixed.

The numerical solution describes the shape of a chain hanging between the chosen endpoints.   The length $L$ of the chain is given by the length of the curve, 
$$
        L = \int_{x_a}^{x_b} \sqrt{1 + (dy/dx)^2} dx
$$
As you might see in a calculus class or in a classical mechanics course.

## Problem 3

Now modify the code to include the numerical integration for the length of the chain. 
Run your code with various values of the constant $k$, say, $1,2,\ldots,8$. Use the resulting data to create a graph of $L$ versus $k$.

Note: You can approximate the derivative $dy/dx$ at the interior nodes using a centered stencil. How should you approximate the 
derivative $dy/dx$ at the endpoints? You could use a one sided stencil. Alternatively, you can avoid this issue by using 
an "open" integration rule. Let $F$ denote the 
integrand with values $F_i$ at the nodes $i=0,\ldots N$. A simple open formula for the integral is given by 
$$
        \int_{x_0}^{x_N} F(x)\,dx \approx h\left[ \frac{3}{2} F_1 + F_2 + \cdots + F_{N-2} + \frac{3}{2} F_{N-1}\right] \ .
$$

In [ ]:
###user defined functions. Make sure to include def chain and comment about each step of chainLen

# We use open formula: replacing ends with half of neighbor points
def chainLen(ychain, dx):
    mysum= (np.sqrt(1 + ((ychain[2]-ychain[0])/(2*dx))**2) + np.sqrt(1 + ((ychain[-1]-ychain[-3])/(2*dx))**2))/2
    for i in range(1, len(y)-1): # Notice that we skip the bound (two ends)
        mysum+= np.sqrt(1 + ((ychain[i+1]-ychain[i-1])/(2*dx))**2)
    return mysum*dx


nodeN= ####FILL IN THE BLANK # numbers of nodes
trialN = ####FILL IN THE BLANK # max number of trials
springs = np.arange(10)+1     #what does this accomplish?
length = np.zeros(len(springs)) #why is this included?

x= np.linspace(0, 1, nodeN+1) # base of N. Notice that there are 101 nodes if including boudnary
h= x[1]-x[0] # increment

for j in range(len(springs)):
    y= ####FILL IN THE BLANK # initial guess 
    for i in range(trialN):
        y[1:-1]= chain(y[2:], y[:-2], h, springs[j])  #why?
    length[j] = chainLen(y,h)

py.plot(springs, length, 'o-')
py.xlim([0, 1.1*springs[-1]])
py.ylim([0, 1.1*length[-1]])
py.xlabel('spring constant, k')
py.ylabel('length of chain')
py.show()

And now, how does the value of k affect the shape of the chain? 

##Problem 4


In [ ]:
# BVP for chain hanging. Equation is y''= k (1+y'^2)^0.5


###User defined functions: use chain, chainLen, BVP_taxi

# Define BVP with taxicab. COMMENT THIS THROUGH. 
def BVP_taxi(yBVP, dx, spring):
  taxicab= 1
  ybuff= np.zeros(len(yBVP)) # initialize ybuff
  while taxicab > 0.005:
    for k in range(len(yBVP)):
        ybuff[k] = yBVP[k];
    yBVP[1:-1]= chain(yBVP[2:], yBVP[:-2], dx, spring)
    taxicab= np.sum(abs(ybuff-yBVP))
  return yBVP


##########################
# Main code
##########################
# Physics setup, copy from previous


#new parameters we're working with:
k= np.linspace(1, 8, 8) # spring constant. why did we use linspace?
cLength= np.zeros(len(k)) # chain length

# Compare length to k. comment this through!!!
for ik, value in enumerate(k):
  y= ####FILL IN THE BLANK # initial guess
  y = BVP_taxi(y, h, value) # BVP
  py.plot(x, y, 'o', label= 'k= '+str(value))
  cLength[ik]= chainLen(y, h) # Calculate length


py.xlabel('x (unit length)')
py.ylabel('height (unit length)')
py.title('Chain shape under different k')
py.legend(bbox_to_anchor=(1.05, 1))
py.show()


# Plot result
py.xlabel('k (1/length)')
py.ylabel('chain length (unit length)')
py.title('Chain length vs curvature constant k')
py.plot(k, cLength, 'x')
py.show()